# SEC Form 4 Analysis

In [78]:
import pandas as pd
import plotly.express as px

In [79]:
form4Data = pd.read_csv("form4.csv")
print(form4Data.columns)
print(form4Data.dtypes)
print(len(form4Data), "rows")
print(form4Data.head(10))

Index(['ISSUER_CIK', 'REPORTER_CIK', 'ACCESSION_NUMBER',
       'NAME_OF_REPORTING_PERSON', 'A_OR_D', 'AMOUNT', 'PRICE',
       'TRANSACTION_DATE', 'TITLE_OF_SECURITY', 'ISSUER_NAME', 'ISSUER_TICKER',
       'IS_DIRECTOR', 'IS_OFFICER', 'IS_TEN_PERCENT_OWNER',
       'IS_OTHER_RELATIONSHIP', 'NEW_AMOUNT_OWNED',
       'DIRECT_OR_INDIRECT_OWNERSHIP'],
      dtype='object')
ISSUER_CIK                        int64
REPORTER_CIK                      int64
ACCESSION_NUMBER                  int64
NAME_OF_REPORTING_PERSON         object
A_OR_D                           object
AMOUNT                          float64
PRICE                           float64
TRANSACTION_DATE                 object
TITLE_OF_SECURITY                object
ISSUER_NAME                      object
ISSUER_TICKER                    object
IS_DIRECTOR                      object
IS_OFFICER                       object
IS_TEN_PERCENT_OWNER             object
IS_OTHER_RELATIONSHIP            object
NEW_AMOUNT_OWNED         

In [80]:
def genSECURL(cik, accessionNumber):
  return "https://sec.gov/Archives/edgar/data/"+cik+"/"+accessionNumber.zfill(18)+"/"+accessionNumber[:-8].zfill(10)+"-"+accessionNumber[-8:-6]+"-"+accessionNumber[-6:]+".txt"

In [81]:
# Get the percentages of each filing added into its own column

def getPercentOfHolding(amount, newAmount):
  if amount > newAmount:
    return (newAmount / amount) * 100
  else:
    return (amount/newAmount) * 100

# form4Data['trade_percent_holding'] = (form4Data['AMOUNT']/form4Data['NEW_AMOUNT_OWNED']) * 100
# form4Data['trade_percent_holding'] = getPercentOfHolding(form4Data['AMOUNT'], form4Data['NEW_AMOUNT_OWNED'])
for row in form4Data.iterrows():
  perc = getPercentOfHolding(row[1]['AMOUNT'], row[1]['NEW_AMOUNT_OWNED'])
  form4Data.at[row[0], 'perc_holding'] = perc
print(form4Data.head(10))
print(genSECURL(str(form4Data["ISSUER_CIK"].iloc[3]), str(form4Data["ACCESSION_NUMBER"].iloc[3])))

   ISSUER_CIK  REPORTER_CIK  ACCESSION_NUMBER NAME_OF_REPORTING_PERSON A_OR_D  \
0     1000623       1703841   106299322009213               Levi Marco      A   
1     1001614       1755918   175591822000002           Riley Philip A      A   
2     1003078       1348379   100307822000048          Jones Douglas E      A   
3     1452857       1547324    92189522001052          WGL Capital LLC      D   
4       59440       1005790    89924322013177          LEBOW BENNETT S      D   
5     1371451       1006020   100602022000001        FORSYTHE GERALD R      A   
6     1012100       1733486   112760222010850       Adefioye Elizabeth      A   
7      700565       1012163   112760222010835             SPARKS RAY A      A   
8      872589       1013064   188385022000031            VAGELOS P ROY      D   
9     1013237       1611324   101323722000055    Snow Frederick Philip      A   

         AMOUNT      PRICE TRANSACTION_DATE  \
0  8.630000e+02    27.5000       2022-04-01   
1  5.000000e+0